In [ ]:
!pip install bert-for-tf2

     |████████████████████████████████| 41 kB 184 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=1fd83378bec880d903070efa6580ae1bd20ed48510b51d16479af79d1ef185c2
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=2bf57add93f1491bdd79c967706838e1720a4c5cc674f60aef0712a2d7d64cce
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=e94411f1ea9a74908cf1874fd0197e646ccd34e430d524910b9714d44c2397f1
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/MyDrive/app") 

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import seaborn as sns
from matplotlib import pyplot as plt
from pandas import DataFrame
from tensorflow.keras import layers
import bert
import pandas as pd
import numpy as np
import re
import random
import math
import os
import joblib

# INFJ/ESTP

class TEXT_MODEL(tf.keras.Model):

    def __init__(self,
           vocabulary_size,
           embedding_dimensions=128,
           cnn_filters=50,
           dnn_units=512,
           model_output_classes=2,
           dropout_rate=0.1,
           training=False,
           name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)

        self.embedding = layers.Embedding(vocabulary_size,
                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                         kernel_size=2,
                         padding="valid",
                         activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                         kernel_size=3,
                         padding="valid",
                         activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                         kernel_size=4,
                         padding="valid",
                         activation="relu")
        self.pool = layers.GlobalMaxPool1D()

        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                             activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                             activation="softmax")

    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l)
        l_1 = self.pool(l_1)
        l_2 = self.cnn_layer2(l)
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3)

        concatenated = tf.concat([l_1, l_2, l_3], axis=-1)  # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)

        return model_output


def tokenize_text(text_input):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_input))


if __name__ == '__main__':

    # hyper parameters
    BATCH_SIZE = 128
    EMB_DIM = 300
    CNN_FILTERS = 100
    DNN_UNITS = 256
    OUTPUT_CLASSES = 10
    DROPOUT_RATE = 0.5
    NB_EPOCHS = 20
    max_len = 2000

    # raw data

    data_set = pd.read_csv("mbti.csv")
    y_4axis = [[], [], [], []]
    text = []
    personality_type = ['IE', 'NS', 'FT', 'JP']
    for _i in range(len(data_set)):
        _text = data_set["posts"][_i]
        _text = _text[1:-1]
        _text = re.sub(r'https?:\/\/.*?[\s+]', ' ', _text)
        _text = re.sub(r'http?:\/\/.*?[\s+]', ' ', _text)
        _text = _text.replace('...|||', ' ')
        _text = _text.replace('|||', ' ')
        text.append(_text)
        for _ in range(4):
          y_4axis[_].append(0 if data_set["type"][_i][_] == personality_type[_][0] else 1)

    # Creating a BERT Tokenizer
    BertTokenizer = bert.bert_tokenization.FullTokenizer

    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
    vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

    # Tokenize all the text
    tokenized_text = [tokenize_text(i) for i in text]

    for _i in range(4):
      # Prerparing Data For Training
      text_with_len = [[text, y_4axis[_i][i], len(text)]
                for i, text in enumerate(tokenized_text)]
      random.shuffle(text_with_len)
      # text_with_len.sort(key=lambda x: x[2])
      # sorted_text_labels = [(text_lab[0], text_lab[1]) for text_lab in text_with_len]
      sorted_text_labels = [(text_lab[0][:max_len], text_lab[1]) for text_lab in text_with_len]
      processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_text_labels, output_types=(tf.int32, tf.int32))
      # batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
      batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((max_len,), ()))

      TOTAL_BATCHES = math.ceil(len(sorted_text_labels) / BATCH_SIZE)
      TEST_BATCHES = TOTAL_BATCHES // 20
      batched_dataset.shuffle(TOTAL_BATCHES)
      test_data = batched_dataset.take(TEST_BATCHES)
      train_data = batched_dataset.skip(TEST_BATCHES)

      VOCAB_LENGTH = len(tokenizer.vocab)
      text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                   embedding_dimensions=EMB_DIM,
                   cnn_filters=CNN_FILTERS,
                   dnn_units=DNN_UNITS,
                   model_output_classes=OUTPUT_CLASSES,
                   dropout_rate=DROPOUT_RATE)

      if OUTPUT_CLASSES == 2:
          text_model.compile(loss="binary_crossentropy",
                    optimizer="adam",
                    metrics=["accuracy"])
      else:
          text_model.compile(loss="sparse_categorical_crossentropy",
                    optimizer="adam",
                    metrics=["sparse_categorical_accuracy"])

      text_model.fit(train_data, epochs=NB_EPOCHS)
      # text_model.fit(train_data, epochs=NB_EPOCHS,validation_data=test_data)
      # test test data
      results = text_model.evaluate(test_data)
      print(f'{personality_type[_i][0]}/{personality_type[_i][1]} Trained Successfully!\n Accuracy: {results[1] * 100}%')

      joblib.dump(text_model, f'{personality_type[_i][0]}{personality_type[_i][1]}.pkl')
      print(f'model {personality_type[_i][0]}/{personality_type[_i][1]} saved.')


Epoch 1/20
65/65 [==============================] - 17s 250ms/step - loss: 0.7068 - sparse_categorical_accuracy: 0.7437
Epoch 2/20
65/65 [==============================] - 17s 254ms/step - loss: 0.4172 - sparse_categorical_accuracy: 0.8104
Epoch 3/20
65/65 [==============================] - 17s 256ms/step - loss: 0.2904 - sparse_categorical_accuracy: 0.8802
Epoch 4/20
65/65 [==============================] - 16s 252ms/step - loss: 0.1719 - sparse_categorical_accuracy: 0.9398
Epoch 5/20
65/65 [==============================] - 16s 250ms/step - loss: 0.0727 - sparse_categorical_accuracy: 0.9812
Epoch 6/20
65/65 [==============================] - 16s 250ms/step - loss: 0.0280 - sparse_categorical_accuracy: 0.9938
Epoch 7/20
65/65 [==============================] - 17s 254ms/step - loss: 0.0097 - sparse_categorical_accuracy: 0.9994
Epoch 8/20
65/65 [==============================] - 17s 255ms/step - loss: 0.0061 - sparse_categorical_accuracy: 0.9995
Epoch 9/20
65/65 [======================

INFO:tensorflow:Assets written to: ram://daad526a-dff3-4930-a673-f2a9f68423b0/assets


model I/E saved.
Epoch 1/20
65/65 [==============================] - 17s 254ms/step - loss: 0.5620 - sparse_categorical_accuracy: 0.8421
Epoch 2/20
65/65 [==============================] - 17s 257ms/step - loss: 0.3518 - sparse_categorical_accuracy: 0.8655
Epoch 3/20
65/65 [==============================] - 17s 255ms/step - loss: 0.2454 - sparse_categorical_accuracy: 0.9004
Epoch 4/20
65/65 [==============================] - 16s 252ms/step - loss: 0.1401 - sparse_categorical_accuracy: 0.9496
Epoch 5/20
65/65 [==============================] - 16s 252ms/step - loss: 0.0686 - sparse_categorical_accuracy: 0.9791
Epoch 6/20
65/65 [==============================] - 17s 253ms/step - loss: 0.0311 - sparse_categorical_accuracy: 0.9910
Epoch 7/20
65/65 [==============================] - 17s 254ms/step - loss: 0.0208 - sparse_categorical_accuracy: 0.9942
Epoch 8/20
65/65 [==============================] - 17s 256ms/step - loss: 0.0040 - sparse_categorical_accuracy: 0.9995
Epoch 9/20
65/65 [=====

INFO:tensorflow:Assets written to: ram://b5e1ba11-bb99-4588-b66d-1dcda08dae3e/assets


model N/S saved.
Epoch 1/20
65/65 [==============================] - 18s 256ms/step - loss: 0.8209 - sparse_categorical_accuracy: 0.5723
Epoch 2/20
65/65 [==============================] - 17s 257ms/step - loss: 0.4191 - sparse_categorical_accuracy: 0.8139
Epoch 3/20
65/65 [==============================] - 17s 254ms/step - loss: 0.2599 - sparse_categorical_accuracy: 0.9015
Epoch 4/20
65/65 [==============================] - 17s 253ms/step - loss: 0.1418 - sparse_categorical_accuracy: 0.9592
Epoch 5/20
65/65 [==============================] - 17s 254ms/step - loss: 0.0674 - sparse_categorical_accuracy: 0.9838
Epoch 6/20
65/65 [==============================] - 17s 253ms/step - loss: 0.0469 - sparse_categorical_accuracy: 0.9863
Epoch 7/20
65/65 [==============================] - 17s 254ms/step - loss: 0.0406 - sparse_categorical_accuracy: 0.9866
Epoch 8/20
65/65 [==============================] - 17s 254ms/step - loss: 0.0325 - sparse_categorical_accuracy: 0.9899
Epoch 9/20
65/65 [=====

INFO:tensorflow:Assets written to: ram://e2279a2f-efe8-41fe-8e73-b6592bd6e7ea/assets


model F/T saved.
Epoch 1/20
65/65 [==============================] - 17s 255ms/step - loss: 0.8363 - sparse_categorical_accuracy: 0.5589
Epoch 2/20
65/65 [==============================] - 17s 255ms/step - loss: 0.5535 - sparse_categorical_accuracy: 0.7160
Epoch 3/20
65/65 [==============================] - 17s 254ms/step - loss: 0.3856 - sparse_categorical_accuracy: 0.8314
Epoch 4/20
65/65 [==============================] - 17s 253ms/step - loss: 0.2267 - sparse_categorical_accuracy: 0.9227
Epoch 5/20
65/65 [==============================] - 17s 253ms/step - loss: 0.0910 - sparse_categorical_accuracy: 0.9779
Epoch 6/20
65/65 [==============================] - 17s 253ms/step - loss: 0.0320 - sparse_categorical_accuracy: 0.9948
Epoch 7/20
65/65 [==============================] - 17s 254ms/step - loss: 0.0119 - sparse_categorical_accuracy: 0.9988
Epoch 8/20
65/65 [==============================] - 17s 255ms/step - loss: 0.0046 - sparse_categorical_accuracy: 1.0000
Epoch 9/20
65/65 [=====

3/3 [==============================] - 0s 101ms/step - loss: 0.7758 - sparse_categorical_accuracy: 0.8229
J/P Trained Successfully!
 Accuracy: 82.29166865348816%
INFO:tensorflow:Assets written to: ram://1710afe2-d780-4712-86d9-c001d6c0d4bf/assets


INFO:tensorflow:Assets written to: ram://1710afe2-d780-4712-86d9-c001d6c0d4bf/assets


model J/P saved.


# 新段落